## Загрузка данных

In [146]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [147]:
df = pd.read_csv('/datasets/users_behavior.csv')

Посмотрим на данные в датасете

In [148]:
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
2252,133.0,949.36,0.0,19036.43,1
1145,43.0,223.90,12.0,23090.88,0
821,56.0,392.72,0.0,6810.52,0
595,107.0,692.17,0.0,15423.29,0
245,65.0,471.52,7.0,15415.81,0


In [149]:
df.isna().mean().apply('{:.2%}'.format)

calls       0.00%
minutes     0.00%
messages    0.00%
mb_used     0.00%
is_ultra    0.00%
dtype: object

Пропусков в данных нет, признаков в текстовом виде тоже

## Разделение данных на выборки

Разделим df на две части - признаки и целевая переменная

In [150]:
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']

Разобьем данные на 3 части - обучающую, тестовую и валидационную выборки

In [151]:
X_train, X_other, y_train, y_other = train_test_split(features,target, train_size=0.6)
X_valid, X_test, y_valid, y_test = train_test_split(X_other, y_other, test_size=0.5)

## Исследование моделей

Используем три различных модели:

In [152]:
logistic_clf = LogisticRegression(random_state=42)
logistic_clf.fit(X_train, y_train)
logistic_predictions = logistic_clf.predict(X_valid)
logistic_accuracy = accuracy_score(y_valid, logistic_predictions)
print('Значение метрики accuracy для модели LogisticRegression: \n', logistic_accuracy)

Значение метрики accuracy для модели LogisticRegression: 
 0.7091757387247278


In [190]:
best_random_forest_model = None
best_random_forest_result = 0
best_rf_est = 0
best_rf_depth = 0
for est in range(10, 51, 5):
    for depth in range (1, 20):
        random_forest_model = RandomForestClassifier(random_state=42, n_estimators=est, max_depth=depth)
        random_forest_model.fit(X_train, y_train)
        random_forest_predictions = random_forest_model.predict(X_valid)
        result = accuracy_score(y_valid, random_forest_predictions)
        if result > best_random_forest_result:
            best_random_forest_model = random_forest_model
            best_random_forest_result = result
            best_rf_est = est
            best_rf_depth = depth
print('Значение метрики accuracy для модели RandomForestClassifier:\n', best_random_forest_result)
print('Лучшие параметры для модели RandomForestClassifier:\n', 'n_estimators:', best_rf_est, 'max_depth:', best_rf_depth)

Значение метрики accuracy для модели RandomForestClassifier:
 0.8040435458786936
Лучшие параметры для модели RandomForestClassifier:
 n_estimators: 40 max_depth: 9


In [191]:
best_decision_tree_model = None
best_decision_tree_result = 0
best_dt_depth = 0
for depth in range (1, 20):
    decision_tree_model = DecisionTreeClassifier(random_state=42, max_depth=depth)
    decision_tree_model.fit(X_train, y_train)
    decision_tree_predictions = decision_tree_model.predict(X_valid)
    result = accuracy_score(y_valid, decision_tree_predictions)
    if result > best_decision_tree_result:
        best_decision_tree_model = decision_tree_model
        best_decision_tree_result = result
        best_dt_depth = depth
print('Значение метрики accuracy для модели DecisionTreeClassifier:\n', best_decision_tree_result)
print('Лучшие параметры для модели DecisionTreeClassifier:\n', 'max_depth:', best_dt_depth)

Значение метрики accuracy для модели DecisionTreeClassifier:
 0.7931570762052877
Лучшие параметры для модели DecisionTreeClassifier:
 max_depth: 8


Лучшее значении метрики accuracy у модели RandomForestClassifier. Худшее значение метрики accuracy у модели LogisticRegression, гиперпараметры этой модели мы не меняли.

## Проверка модели на тестовой выборке

Проверим каждую из моделей на тестовой выборке:

In [ ]:
best_random_forest_model.get_params

In [163]:
logistic_test_predictions = logistic_clf.predict(X_valid)
logistic_test_accuracy = accuracy_score(y_test, logistic_test_predictions)
print('Метрика accuracy для модели LogisticRegression на тестовой выборке:\n', best_decision_tree_result)

Метрика accuracy для модели LogisticRegression на тестовой выборке:
 0.7931570762052877


In [164]:
random_forest_test_predictions = best_random_forest_model.predict(X_test)
random_forest_test_accuracy = accuracy_score(y_test, random_forest_test_predictions)
print('Метрика accuracy для модели RandomForestClassifier на тестовой выборке:\n', random_forest_test_accuracy)

Метрика accuracy для модели RandomForestClassifier на тестовой выборке:
 0.8118195956454122


In [165]:
decision_tree_test_predictions = best_decision_tree_model.predict(X_test)
decision_tree_test_accuracy = accuracy_score(y_test, decision_tree_test_predictions)
print('Метрика accuracy для модели DecisionTreeClassifier на тестовой выборке:\n', decision_tree_test_accuracy)

Метрика accuracy для модели DecisionTreeClassifier на тестовой выборке:
 0.776049766718507


На тестовой выборке лучше всего показала себя модель RandomForestClassifier.  
При этом модель LogisticRegression показала себя лучше, чем DecisionTreeClassifier. В случае с валидационной выборкой ситуация была обратная. 

## Проверка модели на адекватность

Вычислим accuracy для модели, которая предсказывает ответы случайным образом:

In [186]:
print('Accuracy для случайной модели:', (181 *0.5 + 462 * 0.5) / 643)

Accuracy для случайной модели: 0.5


Использованные ранее модели показали лучшие результаты, чем случайная модель. Можем сделать вывод о адекватности наших моделей.

Лучшей моделью для решения нашей задачи можем считать модель RandomForestClassifier со следующими параметрами:

In [199]:
print('n_estimators:', best_rf_est, 'max_depth:', best_rf_depth)

n_estimators: 40 max_depth: 9


Значение accuracy для данной модели на валидационной выборке:

In [200]:
print('Accuracy:', random_forest_test_accuracy)

Accuracy: 0.8118195956454122


## Выводы

Сравнили несколько моделей и выбрали модель с лучшим значением метрики, а также проверили модель на адекватность.